In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import math
import matplotlib.pyplot as plt
import matplotlib.pylab as plt 
%matplotlib inline
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 4
import tensorflow as tf
periodo_aprendizado = '2018-01-05 21:00:00' #periodo incrementado



In [20]:
start_date = '2012-01-01'
parser = lambda x: pd.datetime.strptime(x, "%Y.%m.%d %H:%M:%S")
df = pd.read_csv('EURUSD.csv', sep=',', header=0,parse_dates=['time'],date_parser=parser)
avanco_da_rede = 6099
indice_total_treino = 0


features = 4
timestamp = 2
epochs = 50
janela = 7
x_train = []
y_train = []
x_train = None
y_train = None

mask = (df['time'] > start_date) & (df['time'] <= periodo_aprendizado )
df_train = df.loc[mask]
df_train_orig_2 = df_train.iloc[:,0:5]
df_train = df_train.iloc[:,1:5]
df_train_orig = df_train.copy()
# print(df_train)

indice_total_treino = len(df_train)

df_test      = df.iloc[ indice_total_treino:indice_total_treino+avanco_da_rede,1:5]
df_test_orig = df.iloc[ indice_total_treino:indice_total_treino+avanco_da_rede,0:5]

print(df_test)

# print(df_test_orig)
#periodo_aprendizado = df.iloc[indice_total_treino+avanco_da_rede-3,0]

#Escala nas features
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
df_train = sc.fit_transform(df_train)
#Formação do x_train
x_train = []
y_train = []
length = len(df_train)
for i in range(timestamp, length):
  x_train.append( df_train[i-timestamp:i,:])
  y_train.append( df_train[i,features-1])

x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train[0])

x_train = np.reshape(x_train, (x_train.shape[0], timestamp, features))
  
# # ###Treinamento da rede neural
model = None
model = Sequential()
# model.add(LSTM(units = 480, return_sequences = True, input_shape=( timestamp , features) ) ) 
model.add(LSTM(units = 480, return_sequences = False)) 
#model.add(Dropout(0.1))
#model.add(Dense(units = 1)) 
#model.compile(optimizer = 'adam', loss = 'mean_squared_error')
#model.add(LSTM(50, input_shape=(44725, x_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
model.fit(x_train, y_train, epochs = epochs)
#model.save('drive/My Drive/TCC/modelo/LSTM')
#import tensorflow as tf
#model = tf.keras.models.load_model('drive/My Drive/TCC/modelo/LSTM')
#df_test_close = df_test_close.reshape(-1, 1)
#df_test = sc.fit_transform(df_test)
from sklearn.preprocessing import MinMaxScaler
sc1 = MinMaxScaler()
sc1.fit(df_train_orig.iloc[:,features-1].values.reshape(-1,1))
df_test = sc.transform(df_test)
x_test =    []
y_test =    []
y_max  =    []
y_min  =    [] 
open_test = []
data_test = []

####Teste da rede neural##################

#pegando maximos e minimos para testes com a previsao


length = len(df_test)
for i in range(timestamp, length):
  x_test.append( df_test[i-timestamp:i,:])
  y_test.append( df_test[i,features-1])
  y_max.append( df_test_orig.iloc[i,3])
  y_min.append( df_test_orig.iloc[i,2])
  open_test.append( df_test_orig.iloc[i-1,4] )
  data_test.append( df_test_orig.iloc[i,0])


x_test = np.array(x_test)
y_test = np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], timestamp, features))

#Criando variavel Y_pred para receber as previsões
y_pred = []
y_pred = model.predict(x_test)

#Invertendo a escala para valores reais
predicted_price = sc1.inverse_transform(y_pred)
y_test = sc1.inverse_transform(y_test.reshape(-1,1))

shift = y_test[0] - predicted_price[0]
predicted_price = predicted_price + shift



janela = len(predicted_price)

previsao = pd.DataFrame(data=predicted_price , columns=['Previsão'])
numero_real = pd.DataFrame(data=y_test , columns=['Fecahmaneto'])
preco_de_abertura = pd.DataFrame(data=open_test , columns=['Abertura'])
maxima = pd.DataFrame(data=y_max , columns=['Máxima'])
minima = pd.DataFrame(data=y_min , columns=['Mínima'])
# frames = [ preco_de_abertura, maxima, minima, numero_real, previsao,resumo_acumulado]
# result = pd.concat(frames, axis=1, join='inner')
data_test_pd =  pd.DataFrame(data=data_test , columns=['Data'])
frames_to_csv= [ data_test_pd,maxima, minima, previsao]
result_to_csv = pd.concat( frames_to_csv, axis=1, join='inner')

result_to_csv.to_csv('previsoes_anuais.csv', mode='a', header=False,index=False)




         open1    high1     low1   close1
37293  1.20472  1.20386  1.20493  1.20416
37294  1.20417  1.20278  1.20421  1.20305
37295  1.20272  1.20265  1.20358  1.20315
37296  1.20315  1.20315  1.20389  1.20370
37297  1.20369  1.20328  1.20448  1.20425
...        ...      ...      ...      ...
43387  1.14482  1.14459  1.14657  1.14521
43388  1.14520  1.14383  1.14585  1.14571
43389  1.14571  1.14437  1.14571  1.14461
43390  1.14461  1.14264  1.14485  1.14349
43391  1.14349  1.14254  1.14524  1.14336

[6099 rows x 4 columns]
[[0.71779447 0.7189345  0.71210908 0.71769273 0.71620993]
 [0.71724004 0.71976606 0.71155704 0.71658347 0.71527911]]


ValueError: cannot reshape array of size 372910 into shape (37291,2,4)

In [ ]:

#Calculos  para previsao
previsao = pd.DataFrame(data=predicted_price , columns=['Previsão'])
numero_real = pd.DataFrame(data=y_test , columns=['Fecahmaneto'])
#y_max_pd = pd.DataFrame(data=y_max, columns=['Máxima'])
preco_de_abertura = pd.DataFrame(data=open_test , columns=['Abertura'])
maxima = pd.DataFrame(data=y_max , columns=['Máxima'])
minima = pd.DataFrame(data=y_min , columns=['Mínima'])
frames = [ preco_de_abertura, maxima, minima, numero_real, previsao]
result = pd.concat(frames, axis=1, join='inner')

#Graficos de previsao
plt.plot(y_test[:janela], color='red', label='Preço Atual das Ações')
plt.plot(result.index.values[:janela], result.iloc[:janela,4].values , '.')
plt.fill_between(result.index.values[:janela], result.iloc[:janela,1].values, result.iloc[:janela,2], alpha=0.5)
plt.title('Previsão de Preço de Ações')
plt.xlabel('Tempo')
plt.ylabel('Preço das Ações')
plt.legend()
plt.show()


#####Calculo de acertos da previsao####################
acumulador = 0
lucro_prejuizo = []
resumo_acumulado = []
acertos = 0
erros   = 0
resumo  = []
portcentagem = 0

for i in range (0, janela):
    soma = 0
    if predicted_price[i] in pd.Interval(left=y_min[i], right=y_max[i]):
      soma = abs( np.round(predicted_price[i],5) - np.around(open_test[i], 5 ))
      soma = int(soma*100000)
      acertos = acertos + 1
    else:
      if y_test[i] > open_test[i]:
        if predicted_price[i] > open_test[i]:
          soma = abs( np.round(y_test[i], 5) - np.round(open_test[i], 5) )
          soma = int(soma*100000)
          acertos = acertos + 1
        else:
          soma = np.round(open_test[i], 5) - np.round(y_test[i], 5)
          soma = int(soma*100000)
          erros = erros + 1
      else:
        if predicted_price[i] < open_test[i]:
          soma = abs( np.round(y_test[i], 5) - np.round(open_test[i], 5))
          soma = int(soma*100000)
          acertos = acertos + 1
        else:
          soma = np.round(y_test[i], 5) - np.round(open_test[i], 5)
          soma = int(soma*100000)
          erros = erros + 1
    acumulador = acumulador + soma
    resumo_acumulado.append(acumulador)

portcentagem = acertos*100/janela
print("Acertos: ", acertos)
print("Erros: ", erros)
print("Pontos Ganhos: ", acumulador)
print("%", portcentagem)
print("Meses em teste estudados: ", int(len(predicted_price)/janela))
# resumo = pd.DataFrame(data=lucro_prejuizo , columns=['Resumo'])
resumo_acumulado = pd.DataFrame(data=resumo_acumulado , columns=['Resumo Acumulado '])
#return acumulador



plt.plot(resumo_acumulado, color='blue', label='Preço Atual das Ações')
plt.title('Resumo Acumulado')
plt.xlabel('Tempo')
plt.ylabel('Preço das Ações')
plt.legend()
plt.show()
plt.title('Preço final')
plt.xlabel('Tempo')
plt.ylabel('Preço das Ações')
plt.plot(y_test, color='red', label='Preço Atual das Ações')
plt.show()

In [ ]:
maior_ponto   =  0
rotacao       =  0
maior_acerto      =  0

#for epochs in range(30,50):
for i in range(1):
   model = treinamento_rede_neural(timestamp, features, x_train, y_train, epochs)
   pontos,acerto,data_test = teste_rede(model ,df_test, df_test_orig, sc, janela, shift )
   if acerto > maior_acerto:
     if pontos > maior_ponto:
       #model.save('drive/My Drive/TCC/modelo/LSTM')
       save_tocsv(data_test)
       maior_ponto = pontos
       maior_acerto = acerto

print(maior_ponto, maior_acerto)